In [1]:
import time
import threading
import tkinter as tk
from tkinter import font
from tkinter import messagebox
from playsound import playsound

In [2]:
class PomodoroApp:
    def __init__(self, root):
        #  🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅
        self.root = root
        self.root.title("My 🍅 Timer")
        self.is_running = False # initialize app with timer off
        self.is_break = True
        self.work_mins = 25 # default work duration in minutes
        self.break_mins = 5 # default break duration in minutes
        self.time_left = self.work_mins * 60 # convert to seconds

        # Set starting window size (width x height)
        self.root.geometry("250x180")  # Width = 400, Height = 300

        # Create work timer input
        self.work_label = tk.Label(root, text='Work Time (min):', font=('Comic Sans MS', 14))
        self.work_label.pack(pady=2)
        self.work_entry = tk.Entry(root)
        self.work_entry.pack(pady=2)
        self.work_entry.insert(0, str(self.work_mins)) # default value
        
        # Create break timer input
        self.break_label = tk.Label(root, text='Break Time (min):', font=('Comic Sans MS', 14))
        self.break_label.pack(pady=2)
        self.break_entry = tk.Entry(root)
        self.break_entry.pack(pady=2)
        self.break_entry.insert(0, str(self.break_mins)) # default value
        
        # Create a display label for the timer
        self.timer_label = tk.Label(root, text=self.format_time(self.time_left), font=('Comic Sans MS', 40))
        
        # Create a frame at the bottom to hold the buttons
        self.button_frame = tk.Frame(root)
        self.button_frame.pack(side="bottom", fill='x', pady=10)

        # Create Start, Stop and Reset button
        self.start_button = tk.Button(self.button_frame, text='Start', command=self.start_timer)
        self.start_button.pack(side='left', padx=10)
        
        self.stop_button = tk.Button(self.button_frame, text='Stop', command=self.stop_timer)
        self.stop_button.pack(side='left', padx=10)
        
        self.reset_button = tk.Button(self.button_frame, text='Reset', command=self.reset_timer)
        self.reset_button.pack(side='right', padx=10)
        
    def start_timer(self):
        if not self.is_running:
            # Get user input for the work and break timers
            try:
                self.work_mins = int(self.work_entry.get())
                self.break_mins = int(self.break_entry.get())
                self.time_left = self.work_mins * 60
                # Hide input fields
                self.work_label.pack_forget()
                self.work_entry.pack_forget()
                self.break_label.pack_forget()
                self.break_entry.pack_forget()
                # Show timer label
                self.timer_label.pack(pady=10)
                self.root.geometry("")
                # Start running the timer
                self.is_running = True # set a running flag
                self.root.title(f"Work Timer 🍅")
                self.run_timer()
            except ValueError:
                messagebox.showerror("Invalid input", "Enter only a number for work and break duration, ex.: 10")
            
    def stop_timer(self):
        self.is_running = False
    
    def reset_timer(self):
        # Show input fields again
        self.timer_label.pack_forget() # hide timer
        self.work_label.pack()
        self.work_entry.pack()
        self.break_label.pack()
        self.break_entry.pack()
        self.root.geometry("")
        # Set flag false
        self.stop_timer()
        self.time_left = self.work_mins * 60 # default or user inputed value
        self.timer_label['text'] = self.format_time(self.time_left)
        
    
    def run_timer(self):
        if  self.is_running and self.time_left > 0:
            self.timer_label['text'] = self.format_time(self.time_left)
            self.time_left -= 1
            self.root.after(1000, self.run_timer) # call timer again after 1 second
        elif self.is_break and self.time_left == 0:
            playsound('DarkMeta.wav')
            self.root.title(f"Time for a {self.break_mins} minutes Break!")
            time.sleep(1)
            self.start_break_timer()
        elif not self.is_break and self.time_left == 0:
            self.root.title(f"🍅's up, Good Work!")
            self.is_running = False
    
    def start_break_timer(self):
        self.root.title(f"Break Timer 🍅")
        self.is_break = False
        self.time_left = self.break_mins * 60 # set time for the break
        self.timer_label['text'] = self.format_time(self.time_left)
        self.run_timer() # start the break timer
    
    def format_time(self, time_in_seconds):
        minutes, seconds = divmod(time_in_seconds, 60)
        return f"🍅 {minutes:02d}:{seconds:02d} 🍅"


In [3]:
root = tk.Tk()
app = PomodoroApp(root)
root.mainloop()